# <font size=6 color='green'><center>**IT 496 - Introduction to Data Mining**</center></font>
### **<center>Course Project - 03<br/><br/>ICC Cricket World Cup 2023 ML Challange<center>**

 &emsp; **Group Name : <font color='#0099ff'>DataForce</font>** <br/>
 &emsp; **Group ID : <font color='#0099ff'>T06</font>**

In [1]:
# importing required libraries
import numpy as np
import pandas as pd

### <center>Task 1</center>

**Problems**
We have chosen the following two prediction problems for this task.<br/>
a. Predicting the player with the highest number of wickets taken in the tournament<br/>
b. Predicting the batsman who will score most runs in
the tournament.

---

Task 1a: Predicting the player with highest number of wickets taken

In [3]:
# loading the data
players_data = pd.read_csv('../Dataset/ICC_Player_Stats.csv')
players_data.head()

,Team Name,Player name,Specalist,Span,Matches,Innings,Not Out,Runs,Highest Score,Average,BF,Strike Rate,Half Centuries,Centuries,0,Fours,Sixes,Wickets Taken,Catches
0,India,Rohit Sharma,Batsman,2007-2023,259,251,36,10554,264.0,49.08,11530.0,91.53,31.000000,54.0,16.0,978.0,314.0,8.0,92
1,India,Hardik Pandya,AllRounder,2016-2023,86,61,9,1769,92.0,34.01,1603.0,110.35,3.396552,11.0,4.0,132.0,67.0,84.0,45
2,India,Shubman Gill,Batsman,2019-2023,41,41,6,2136,208.0,61.02,2090.0,102.20,6.000000,11.0,1.0,243.0,45.0,0.0,30
3,India,Virat Kohli,Batsman,2008-2023,289,277,44,13626,183.0,58.48,14565.0,93.55,49.000000,70.0,16.0,1276.0,148.0,4.0,150
4,India,Shreyas Iyer,Batsman,2017-2023,55,50,5,2094,113.0,46.53,2141.0,97.80,3.000000,17.0,1.0,198.0,48.0,0.0,24


In [5]:
# checking for and removing any null values present
players_data.isnull().sum()

Team Name         0
Player name       0
Specalist         0
Span              0
Matches           0
Innings           0
Not Out           0
Runs              0
Highest Score     0
Average           0
BF                0
Strike Rate       0
Half Centuries    0
Centuries         0
0                 0
Fours             0
Sixes             0
Wickets Taken     0
Catches           0
dtype: int64

We had already cleaned and processed this data, there are no null values present

In [7]:
players_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Team Name       150 non-null    object 
 1   Player name     150 non-null    object 
 2   Specalist       150 non-null    object 
 3   Span            150 non-null    object 
 4   Matches         150 non-null    int64  
 5   Innings         150 non-null    int64  
 6   Not Out         150 non-null    int64  
 7   Runs            150 non-null    int64  
 8   Highest Score   150 non-null    float64
 9   Average         150 non-null    float64
 10  BF              150 non-null    float64
 11  Strike Rate     150 non-null    float64
 12  Half Centuries  150 non-null    float64
 13  Centuries       150 non-null    float64
 14  0               150 non-null    float64
 15  Fours           150 non-null    float64
 16  Sixes           150 non-null    float64
 17  Wickets Taken   150 non-null    flo

There are few Nominal attributes like Team Name, Player Name and Specialist and span which are need to be converted to numerical attributes

**Approach**: Our approach is to divide the dataset into 15 parts, and for each part, train the model using other 14 parts and predict the outputs for this single part. This way we predict the outputs for all the 15 parts and take the maximum of that.

In [99]:
# Randomly shuffling the data
players_data = players_data.sample(frac=1)
data_parts = []
N = players_data.shape[0]
for i in range(N//10):
     df = players_data.iloc[10*i:10*(i+1),:]
     data_parts.append(df)

print(len(data_parts))

15


In [100]:
from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [103]:
predictions=[]

# for each data split, training and testing
for i in range(len(data_parts)):
    df1 = data_parts[i]
    rem = [data_parts[j] for j in range(11) if not j == i]
    trainData = pd.concat(rem, axis=0)
    xtrain = comb.drop(['Player name','Span','Wickets Taken'],axis=1)
    ytrain = comb['Wickets Taken']
    xtest= df1.drop(['Player name','Span','Wickets Taken'],axis=1)
    ytest = df1['Wickets Taken']

    # label encoding
    encoder  = LabelEncoder()
    xtrain['Team Name'] = encoder.fit_transform(xtrain['Team Name'])
    xtest['Team Name'] = encoder.transform(xtest['Team Name'])
    xtrain['Specalist'] = encoder.fit_transform(xtrain['Specalist'])
    xtest['Specalist'] = encoder.transform(xtest['Specalist'])

    # normalization
    scaler = RobustScaler()
    ytrain = np.array(ytrain)
    ytest = np.array(ytest)
    xtrain = scaler.fit_transform(xtrain)
    xtest = scaler.transform(xtest)
    ytrain = scaler.fit_transform(ytrain.reshape(-1,1))
    ytest = scaler.transform(ytest.reshape(-1,1))

    # hyperparameter tuning and model fitting
    regressorModel = xgb.XGBRegressor(objective='reg:squarederror',booster='gbtree')
    regressorModel.fit(xtrain,ytrain)
    y_pred= regressorModel.predict(xtest)
    index=0
    for i,row in df1.iterrows():
        predictions.append([row['Player name'], y_pred[index]])
        index+=1

In [104]:
# sort according to wickets taken in descending order
predictions.sort(key=lambda x:-x[1])
predictions[0][0]

'Shakib Al Hasan'

The model predicted that Shakib Al Hasan will be the highest wicket taker in the tournament

**Task 1b**. Predicting the batsman who will score most runs in
the tournament.

Following similar procedure as in the previous task

In [95]:
predictions=[]

# for each data split, training and testing
for i in range(len(data_parts)):
    df1 = data_parts[i]
    rem = [data_parts[j] for j in range(11) if not j == i]
    trainData = pd.concat(rem, axis=0)
    xtrain = comb.drop(['Player name','Span','Runs'],axis=1)
    ytrain = comb['Runs']
    xtest= df1.drop(['Player name','Span','Runs'],axis=1)
    ytest = df1['Runs']

    # label encoding
    encoder  = LabelEncoder()
    xtrain['Team Name'] = encoder.fit_transform(xtrain['Team Name'])
    xtest['Team Name'] = encoder.transform(xtest['Team Name'])
    xtrain['Specalist'] = encoder.fit_transform(xtrain['Specalist'])
    xtest['Specalist'] = encoder.transform(xtest['Specalist'])

    # normalization
    scaler = RobustScaler()
    ytrain = np.array(ytrain)
    ytest = np.array(ytest)
    xtrain = scaler.fit_transform(xtrain)
    xtest = scaler.transform(xtest)
    ytrain = scaler.fit_transform(ytrain.reshape(-1,1))
    ytest = scaler.transform(ytest.reshape(-1,1))

    # hyperparameter tuning and model fitting
    regressorModel = xgb.XGBRegressor(objective='reg:squarederror',booster='gbtree')
    regressorModel.fit(xtrain,ytrain)
    y_pred= regressorModel.predict(xtest)
    index=0
    for i,row in df1.iterrows():
        predictions.append([row['Player name'], y_pred[index]])
        index+=1

In [96]:
# sort according to sixes
predictions.sort(key=lambda x:-x[1])
predictions[0][0]

'Virat Kohli'

The model predicted that Virat Kohli will have the highest runs in the tournament

---

### <center>Task 2</center>
You are required to predict the two finalist teams in the ICC Cricket World Cup 2023 and the
players (11 players for each finalist team) who are likely to be part of these teams. This task
involves team composition prediction.

In [2]:
from teamCompositionPredictor import getTeamCompsition
from tensorflow import keras
from Helpers import getTeamsMapping,getVenuesMapping, getPlayerRole, getTeamStats

In [3]:
def getMatchWinnerPredictor():
  # loading the pre-trained model
  model = keras.models.load_model('MatchWinnerPredictor.keras')
  return model

In [4]:
additional_cols1 = ['team1 avg score','team1 avg wickets','team1 avg catches','team1 avg fours',
                   'team1 avg sixes','team1 odi rank']
additional_cols2 = ['team2 avg score','team2 avg wickets','team2 avg catches','team2 avg fours',
                   'team2 avg sixes','team2 odi rank']

Predicting for the remaining matches of ICC

In [5]:
matches = pd.DataFrame(columns=['season', 'team1', 'team2', 'Venue', 'team1 avg score',
       'team1 avg wickets', 'team1 avg catches', 'team1 avg fours',
       'team1 avg sixes', 'team1 odi rank', 'team2 avg score',
       'team2 avg wickets', 'team2 avg catches', 'team2 avg fours',
       'team2 avg sixes', 'team2 odi rank'])

In [6]:
matches.loc[0,['season','team1','team2','Venue']] = ['2023','India','New Zealand','Wankhede Stadium, Mumbai']
matches.loc[1,['season','team1','team2','Venue']] = ['2023','South Africa','Australia','Eden Gardens, Kolkata']
matches.loc[0,additional_cols1] = getTeamStats(matches.iloc[0]['team1'])
matches.loc[0,additional_cols2] = getTeamStats(matches.iloc[0]['team2'])
matches.loc[1,additional_cols1] = getTeamStats(matches.iloc[1]['team1'])
matches.loc[1,additional_cols2] = getTeamStats(matches.iloc[1]['team2'])
# teams in finals is not yet know
matches.loc[2,['season','Venue']] = ['2023','Narendra Modi Stadium, Ahmedabad']
matches.head()

,season,team1,team2,Venue,team1 avg score,team1 avg wickets,team1 avg catches,team1 avg fours,team1 avg sixes,team1 odi rank,team2 avg score,team2 avg wickets,team2 avg catches,team2 avg fours,team2 avg sixes,team2 odi rank
0,2023,India,New Zealand,"Wankhede Stadium, Mumbai",25.180872,85.37697,38.8,244.555705,55.894855,1.0,18.430865,72.762424,27.533333,121.666667,25.133333,5.0
1,2023,South Africa,Australia,"Eden Gardens, Kolkata",22.147129,64.220606,20.0,121.0,27.2,3.0,23.453834,59.333333,34.6,191.333333,47.266667,2.0
2,2023,NaN,NaN,"Narendra Modi Stadium, Ahmedabad",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
venuesMapping = getVenuesMapping()
teamsMapping = getTeamsMapping()
matches['Venue'] = matches['Venue'].apply(lambda x:venuesMapping[x])
matches.loc[0,'team1'] = teamsMapping[matches.iloc[0]['team1']]
matches.loc[0,'team2'] = teamsMapping[matches.loc[0]['team2']]
matches.loc[1,'team1'] = teamsMapping[matches.loc[1]['team1']]
matches.loc[1,'team2'] = teamsMapping[matches.loc[1]['team2']]

In [8]:
nn_model = getMatchWinnerPredictor()

In [9]:
# importing the RobustScaler 
# to normalize using scaler fitted on train data
import pickle
with open('RobustScaler.pkl','rb') as f:
    scaler = pickle.load(f)

Semi-Final 1 Prediction

In [10]:
input1 = np.asarray(matches.iloc[0])
x1 = scaler.transform(np.expand_dims(input1,axis=0))
nn_model.predict(x1)

1/1 [==============================] - 0s 108ms/step


array([[0.6995439, 0.3004561]], dtype=float32)

The Model predicted that Team 1(India) wins with probability 0.699 and Team 2(New Zealand) wins with probability 0.3.
Thus India is the winner of semi-final 1!!

Semi-Final 2 Prediction

In [11]:
input2 = np.asarray(matches.iloc[1])
x2 = scaler.transform(np.expand_dims(input2,axis=0))
nn_model.predict(x2)

1/1 [==============================] - 0s 27ms/step


array([[0.76192933, 0.23807067]], dtype=float32)

The Model predicted that Team 1(South Africa) wins with probability 0.762 and Team 2(Australia) wins with probability 0.238.
Thus South Africa is the winner of semi-final 2!!

### Team Composition Prediction

In [12]:
team1List, team2List = getTeamCompsition("India","South Africa","Narendra Modi Stadium, Ahmedabad")
print("Playing 11 of India : ")
print(team1List)
print("\nPlaying 11 of South Africa")
print(team2List)

Playing 11 of India : 
['Shubman Gill' 'Virat Kohli' 'Rohit Sharma' 'KL Rahul' 'Shreyas Iyer'
 'Ishan Kishan' 'Mohammed Shami' 'Kuldeep Yadav' 'Jasprit Bumrah'
 'Suryakumar Yadav' 'Hardik Pandya']

Playing 11 of South Africa
['Kagiso Rabada' 'Temba Bavuma (Captain)' 'Quinton de Kock' 'David Miller'
 'Aiden Markram' 'Rassie van der Dussen' 'Reeza Hendricks' 'Lungi Ngidi'
 'Tabraiz Shamsi' 'Heinrich Klaasen' 'Marco Jansen']


---

### <center>Task 3</center>
You should predict the winner of the ICC Cricket World Cup 2023.

In [13]:
matches.loc[2,['team1','team2']]=['India','South Africa']
matches.loc[2,additional_cols1] = getTeamStats(matches.iloc[2]['team1'])
matches.loc[2,additional_cols2] = getTeamStats(matches.iloc[2]['team2'])
matches.loc[2,'team1'] = teamsMapping[matches.iloc[2]['team1']]
matches.loc[2,'team2'] = teamsMapping[matches.loc[2]['team2']]
matches.head()

,season,team1,team2,Venue,team1 avg score,team1 avg wickets,team1 avg catches,team1 avg fours,team1 avg sixes,team1 odi rank,team2 avg score,team2 avg wickets,team2 avg catches,team2 avg fours,team2 avg sixes,team2 odi rank
0,2023,5,7,9,25.180872,85.37697,38.8,244.555705,55.894855,1.0,18.430865,72.762424,27.533333,121.666667,25.133333,5.0
1,2023,9,2,10,22.147129,64.220606,20.0,121.0,27.2,3.0,23.453834,59.333333,34.6,191.333333,47.266667,2.0
2,2023,5,9,1,25.180872,85.37697,38.8,244.555705,55.894855,1.0,22.147129,64.220606,20.0,121.0,27.2,3.0


In [14]:
finalInput = np.asarray(matches.iloc[2])
x = scaler.transform(np.expand_dims(finalInput,axis=0))
nn_model.predict(x)

1/1 [==============================] - 0s 23ms/step


array([[0.82144415, 0.1785559 ]], dtype=float32)

The model predicted 0.82 for Team 1(India) and 0.18 for Team 2(South Africa)

The Model predicted the final winner as India with probability 82%

---